In [39]:
# magic function para hacer que los graficos de matplotlib se renderizen en el notebook.
%matplotlib inline

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv

plt.style.use('default') # haciendo los graficos un poco mas bonitos xD
plt.rcParams['figure.figsize'] = (15, 5)


In [40]:
filesProperati = os.listdir("datos/properati")

In [43]:
dataFrames = []
for file_name in filesProperati:
    try:
        newDataFrame = pd.read_csv("datos/properati/"+file_name)
        dataFrames.append(newDataFrame)
    except:
        #Ver causa de este error (ParserError)
        #Error con el archivo: properati-AR-2015-06-01-properties-sell.csv
        #Error con el archivo: properati-AR-2015-08-01-properties-sell.csv
        #Error con el archivo: properati-AR-2015-04-01-properties-sell.csv
        #Error con el archivo: properati-AR-2015-07-01-properties-sell.csv
        print "Error con el archivo: "+file_name

Error con el archivo: properati-AR-2015-06-01-properties-sell.csv
Error con el archivo: properati-AR-2015-08-01-properties-sell.csv
Error con el archivo: properati-AR-2015-04-01-properties-sell.csv
Error con el archivo: properati-AR-2015-07-01-properties-sell.csv


In [54]:
#Hay algunos archivos de properati que tienen menos informacion, ver que hacer con ellos
masCols = []
for df in dataFrames:
    if (len(df.columns) == 27):
        masCols = df.columns
menosCols = []
for df in dataFrames:
    if (len(df.columns) == 20):
        menosCols = df.columns
diferencias = list(set(masCols) - set(menosCols))
#No tienen la superficie total en m2, asi que para estos archivos, 
#si no tenemos info del precio en alguna columna price, descartamos las filas
print diferencias


['surface_total_in_m2', 'description', 'title', 'surface_covered_in_m2', 'price_per_m2', 'country_name', 'id', 'state_name']


In [61]:
#¿que tipos de propiedades tenemos?
tiposDePropiedades = []
for df in dataFrames:
    try:
        propiedades = df[["property_type"]]
        tiposDePropiedades.append(propiedades)
    except KeyError:
        #No se por que me esta abriendo archivos que no son de properati, por eso este except
        pass

34


In [63]:
tiposDePropiedades = pd.concat(tiposDePropiedades)

In [74]:
tiposDePropiedades["property_type"].value_counts()

apartment    1222108
house         718179
PH            121444
store          37229
Name: property_type, dtype: int64

In [ ]:
#Algo que se podria analizar es ver que tipo de propiedad tiene mayor precio promedio en los
#ultimos años. 